<a href="https://colab.research.google.com/github/PrateekKumar135/COSINE_SIMILARITY/blob/main/COSINE_SIMILARITY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INSTALLING LIBRARIES

In [1]:
!pip install datasets tensorflow tensorflow-text nltk gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 wh

# IMPORTING LIBRARIES

In [2]:
from datasets import load_dataset
import tensorflow as tf
import tensorflow_text as tf_text  # Required for tokenization
import nltk
from nltk.corpus import stopwords
import re
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec

# DATASET LOAD

In [3]:
ds = load_dataset("agentlans/high-quality-english-sentences")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.30k [00:00<?, ?B/s]

train.txt.gz:   0%|          | 0.00/85.5M [00:00<?, ?B/s]

test.txt.gz:   0%|          | 0.00/9.49M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1534699 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/170522 [00:00<?, ? examples/s]

In [4]:
ds

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1534699
    })
    test: Dataset({
        features: ['text'],
        num_rows: 170522
    })
})

**EXTRACT TEXT DATA**

In [5]:
# Extract text data
sentences = ds["train"]["text"][:10000]
sentences[:8]

['Soon we dropped into a living forest, where cold-tolerant evergreens and boreal animals still evoke the Canadian heritage of an ecosystem pushed south by glaciers 20,000 years ago.',
 'Annual population growth rate (2011 est., CIA World Factbook): 1.284%.',
 'This has led to the recent banning of Neonics in the EU, however the US and Canada are still using this chemical pesticide.',
 "In addition, these colors weren't confined to a province but rather irregularly scattered across various regions over all of China.",
 'A family member or a support person may stay with a patient during recovery.',
 'By the time emigrants got to what is now Nevada, they had been on the trail long enough that their supplies were down to just the basics.',
 "Advances in valve technology allow air cannons to operate with lower ... generation, coal, metal and .... installed on a cement plant's primary crusher.",
 'It stretched 363 miles across New York State from the Hudson River in the east to Lake Erie in

**Convert Sentences into TensorFlow Tensors**

In [6]:
# Convert to TensorFlow tensors
sentence_tensors = tf.constant(sentences)

**Perform Word Tokenization**

In [7]:
# Use UnicodeScriptTokenizer for word tokenization
word_tokenizer = tf_text.UnicodeScriptTokenizer()
word_tokens = word_tokenizer.tokenize(sentence_tensors)

**Convert Tokenized Words to List**

In [8]:
# Convert tokenized words to list
word_tokens_list = [tokens.numpy().tolist() for tokens in word_tokens]
word_tokens_list[:5]

[[b'Soon',
  b'we',
  b'dropped',
  b'into',
  b'a',
  b'living',
  b'forest',
  b',',
  b'where',
  b'cold',
  b'-',
  b'tolerant',
  b'evergreens',
  b'and',
  b'boreal',
  b'animals',
  b'still',
  b'evoke',
  b'the',
  b'Canadian',
  b'heritage',
  b'of',
  b'an',
  b'ecosystem',
  b'pushed',
  b'south',
  b'by',
  b'glaciers',
  b'20,000',
  b'years',
  b'ago',
  b'.'],
 [b'Annual',
  b'population',
  b'growth',
  b'rate',
  b'(2011',
  b'est',
  b'.,',
  b'CIA',
  b'World',
  b'Factbook',
  b'):1.284%.'],
 [b'This',
  b'has',
  b'led',
  b'to',
  b'the',
  b'recent',
  b'banning',
  b'of',
  b'Neonics',
  b'in',
  b'the',
  b'EU',
  b',',
  b'however',
  b'the',
  b'US',
  b'and',
  b'Canada',
  b'are',
  b'still',
  b'using',
  b'this',
  b'chemical',
  b'pesticide',
  b'.'],
 [b'In',
  b'addition',
  b',',
  b'these',
  b'colors',
  b'weren',
  b"'",
  b't',
  b'confined',
  b'to',
  b'a',
  b'province',
  b'but',
  b'rather',
  b'irregularly',
  b'scattered',
  b'across',
  b'

**Decode Byte Tokens into Strings**

In [9]:
# Decode byte tokens to strings
decoded_tokens = [[token.decode("utf-8") for token in sentence] for sentence in word_tokens_list]

**Download Required NLTK Resources**

In [10]:
# Download stopwords list if not already downloaded
nltk.download("stopwords")
nltk.download('wordnet')

# Load English stopwords
stop_words = set(stopwords.words("english"))
# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


**Define a Text Cleaning Function**

In [11]:
# Function to apply stopword removal
def remove_stopwords(tokens):
    return [word.lower() for word in tokens if word.lower() not in stop_words]

# Function to apply regex processing
def apply_regex(tokens):
    return [re.sub(r"[^a-zA-Z]", "", word).strip() for word in tokens if re.sub(r"[^a-zA-Z]", "", word).strip()]  # Keep only letters

# Function to apply lemmatization
def apply_lemmatization(tokens):
    return [lemmatizer.lemmatize(word) for word in tokens]

# Apply processing in sequence
processed_sentences = []
for sentence in decoded_tokens:
    stopword_removed = remove_stopwords(sentence)  # Step 1: Remove stopwords
    regex_applied = apply_regex(stopword_removed)  # Step 2: Apply regex cleaning
    lemmatized = apply_lemmatization(regex_applied)  # Step 3: Apply lemmatization

    processed_sentences.append({
        "After Stopword Removal": stopword_removed,
        "After Regex": regex_applied,
        "After Lemmatization": lemmatized
    })

# Display results
for i, process in enumerate(processed_sentences):
    print(f"🔹 Sentence {i+1}")
    print(f"    After Stopword Removal: {process['After Stopword Removal']}")
    print(f"    After Regex: {process['After Regex']}")
    print(f"    After Lemmatization: {process['After Lemmatization']}")
    print("-" * 80)


Streaming output truncated to the last 5000 lines.
🔹 Sentence 9001
    After Stopword Removal: ['birthplace', 'america', "'", 'experimental', '-', 'theater', 'scene', 'fostered', 'writer', 'sam', 'shepard', ',', 'director', 'jerzy', 'grotowski', ',', 'composer', 'elizabeth', 'swados', '.']
    After Regex: ['birthplace', 'america', 'experimental', 'theater', 'scene', 'fostered', 'writer', 'sam', 'shepard', 'director', 'jerzy', 'grotowski', 'composer', 'elizabeth', 'swados']
    After Lemmatization: ['birthplace', 'america', 'experimental', 'theater', 'scene', 'fostered', 'writer', 'sam', 'shepard', 'director', 'jerzy', 'grotowski', 'composer', 'elizabeth', 'swados']
--------------------------------------------------------------------------------
🔹 Sentence 9002
    After Stopword Removal: ['lenin', '(', 'basing', 'position', 'developed', 'earlier', 'kautsky', ',', 'principal', 'theoretician', 'spd', ')', 'insisted', 'socialist', 'consciousness', 'develop', 'spontaneously', 'within', 'w

**CBOW(BAG OF WORDS)**

In [12]:
from gensim.models import Word2Vec

# Prepare the processed text as input for Word2Vec
tokenized_sentences = [process["After Lemmatization"] for process in processed_sentences]
# print(tokenized_sentences[:5])
# Train Word2Vec model
word2vec_model = Word2Vec(
    sentences=tokenized_sentences,  # Input sentences
    vector_size=100,  # Each word will be represented as a n-dimensional vector
    window=3,  # Context window size
    min_count=2,  # Ignore words with frequency less than 2
    sg=0,  # Use CBOW (better for rare words)
    workers=4,  # Use 4 CPU cores
    epochs=20,  # Train for 10 epochs
    hs=1
)

# Save the trained model
word2vec_model.save("word2vec.model")

# Example: Get vector representation of a word
word = "social"
if word in word2vec_model.wv:
    print(f"Vector representation of '{word}':\n{word2vec_model.wv[word]}")
    print(len(word2vec_model.wv[word]))
else:
    print(f"'{word}' not found in vocabulary")

# Example: Find similar words
if word in word2vec_model.wv:
    similar_words = word2vec_model.wv.most_similar(word)
    print(f"\nTop 5 words similar to '{word}':")
    for sim_word, similarity in similar_words:
        print(f"  {sim_word}: {similarity:.4f}")


Vector representation of 'social':
[ 1.01278436e+00 -6.42162979e-01 -1.23449743e+00 -5.20182788e-01
  6.55271411e-01 -9.83836353e-01  7.25462556e-01  3.68366167e-02
  3.01225662e-01 -1.09758735e+00  4.02019292e-01 -5.52700162e-01
 -8.40459645e-01 -4.55762371e-02  4.17709976e-01  8.15138042e-01
  5.60612500e-01 -9.44923341e-01  1.42891929e-01 -7.71663368e-01
  9.71354783e-01 -1.18228650e+00  3.40897143e-01 -6.41768500e-02
 -2.24597979e+00  8.72681856e-01  1.38370112e-01 -7.15166271e-01
  3.89574856e-01 -1.67819035e+00  4.62954164e-01  4.43229109e-01
 -9.28812996e-02  9.86635268e-01  1.12305152e+00  3.24206173e-01
 -2.03651071e-01 -8.13216925e-01  1.24686770e-01 -2.12188697e+00
  4.01429743e-01  3.51380855e-01  1.01603866e+00 -4.47205484e-01
  6.31710231e-01 -5.02178192e-01 -1.03598404e+00 -1.24448931e+00
  1.50407463e-01 -5.58010638e-01 -8.60867739e-01 -4.84465845e-02
  1.32222652e+00 -7.59393036e-01  6.11274600e-01  1.21168137e+00
  8.22969437e-01 -2.31122136e-01  1.33600786e-01 -9.577

**SKIP-GRAM**

In [13]:
from gensim.models import Word2Vec

# Prepare the processed text as input for Word2Vec
tokenized_sentences = [process["After Lemmatization"] for process in processed_sentences]
# print(tokenized_sentences[:5])
# Train Word2Vec model
word2vec_model = Word2Vec(
    sentences=tokenized_sentences,  # Input sentences
    vector_size=100,  # Each word will be represented as a n-dimensional vector
    window=3,  # Context window size
    min_count=2,  # Ignore words with frequency less than 2
    sg=1,  # Use Skip-gram (better for rare words)
    workers=4,  # Use 4 CPU cores
    epochs=20,  # Train for 10 epochs
    hs=1

)

# Save the trained model
word2vec_model.save("word2vec.model")

# Example: Get vector representation of a word
word = "social"
if word in word2vec_model.wv:
    print(f"Vector representation of '{word}':\n{word2vec_model.wv[word]}")
    print(len(word2vec_model.wv[word]))
else:
    print(f"'{word}' not found in vocabulary")

# Example: Find similar words
if word in word2vec_model.wv:
    similar_words = word2vec_model.wv.most_similar(word)
    print(f"\nTop 5 words similar to '{word}':")
    for sim_word, similarity in similar_words:
        print(f"  {sim_word}: {similarity:.4f}")


Vector representation of 'social':
[ 0.64958525  0.5112998  -0.4573714  -0.28526485 -0.10216914 -0.67095363
 -0.14552854  0.04600862 -0.47797123 -0.15101807  0.07563026 -0.6836453
 -0.67107224 -0.52859724 -0.36733413 -0.33429387  0.04230069 -0.40996194
 -0.12791686 -0.5226963  -0.05869799 -0.27390465  0.1392044  -0.25652578
 -0.53674364  0.24603082  0.52525115  0.05088064  0.19028185 -0.46839535
  0.15131186  0.31586507 -0.30870238  1.0558351  -0.17683643 -0.00480261
  0.7669628  -0.45728698  0.54096055 -1.1030493  -0.57941395 -0.02206804
  0.42199555 -0.41206542  0.6897164  -0.06488128 -0.52425784 -0.06905688
  0.26800227 -0.16805112 -0.54853874  0.06217616  0.63021487  0.32168454
 -0.30168545  0.5366433   0.32375062 -0.07221498  0.04280208  0.27604666
 -0.39921156 -0.04079762 -0.02129481 -0.2465361   0.04827606 -0.11124411
  0.63659024 -0.4180326  -0.410827    0.45609152 -0.05449094 -0.16474184
  0.08199838  0.19256836  0.39482927  0.7310616   0.10757067 -0.46182972
  0.01271802  0.0